In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/nyc_temperatures.csv')
df.head()

,date,datatype,station,attributes,value
0,2018-10-01T00:00:00,TAVG,GHCND:USW00014732,"H,,S,",21.2
1,2018-10-01T00:00:00,TMAX,GHCND:USW00014732,",,W,2400",25.6
2,2018-10-01T00:00:00,TMIN,GHCND:USW00014732,",,W,2400",18.3
3,2018-10-02T00:00:00,TAVG,GHCND:USW00014732,"H,,S,",22.7
4,2018-10-02T00:00:00,TMAX,GHCND:USW00014732,",,W,2400",26.1


### Renaming columns

In [3]:
df.columns

Index(['date', 'datatype', 'station', 'attributes', 'value'], dtype='object')

In [4]:
df.rename(
    columns={'value': 'temp_C',
             'attributes': 'flags'},
             inplace=True  # update the original dataframe
)

In [5]:
df.columns

Index(['date', 'datatype', 'station', 'flags', 'temp_C'], dtype='object')

>   
    rename()
    Both Series and Index objects can also be renamed using their rename() methods. Simply pass in the new name

In [6]:
# we can also do columns transformation with rename()
df.rename(str.upper, axis='columns').columns

Index(['DATE', 'DATATYPE', 'STATION', 'FLAGS', 'TEMP_C'], dtype='object')

### Type conversion

In [7]:
df.dtypes

date         object
datatype     object
station      object
flags        object
temp_C      float64
dtype: object

Note that the date column isn't actually being stored as a datetime

In [8]:
df.date = pd.to_datetime(df.date)

In [9]:
df.dtypes

date        datetime64[ns]
datatype            object
station             object
flags               object
temp_C             float64
dtype: object

In [10]:
# Now, we can get useful information when we summarize the date
# column:
df.date.describe()

count                     93
mean     2018-10-16 00:00:00
min      2018-10-01 00:00:00
25%      2018-10-08 00:00:00
50%      2018-10-16 00:00:00
75%      2018-10-24 00:00:00
max      2018-10-31 00:00:00
Name: date, dtype: object

> 
    When working with a DatetimeIndex object, if we need to keep track of time zones, we can use the `tz_localize()` method to associate our datetimes with a time zone

In [11]:
pd.date_range(start='2018-10-25', periods=2, freq='D') \
              .tz_localize('EST')

DatetimeIndex(['2018-10-25 00:00:00-05:00', '2018-10-26 00:00:00-05:00'], dtype='datetime64[ns, EST]', freq=None)

This also works with Series and DataFrame objects that have an index of type DatetimeIndex

We can read in the CSV file again and, this time, specify that the date column will be our index and that we should parse any dates in the CSV file into
datetimes

In [12]:
eastern = pd.read_csv(
    'data/nyc_temperatures.csv',
    index_col='date', parse_dates=True
    ).tz_localize('EST')

eastern.head()

,datatype,station,attributes,value
date,,,,
2018-10-01 00:00:00-05:00,TAVG,GHCND:USW00014732,"H,,S,",21.2
2018-10-01 00:00:00-05:00,TMAX,GHCND:USW00014732,",,W,2400",25.6
2018-10-01 00:00:00-05:00,TMIN,GHCND:USW00014732,",,W,2400",18.3
2018-10-02 00:00:00-05:00,TAVG,GHCND:USW00014732,"H,,S,",22.7
2018-10-02 00:00:00-05:00,TMAX,GHCND:USW00014732,",,W,2400",26.1


We can use the `tz_convert()` method to change the time zone into a different one

In [13]:
eastern.tz_convert('UTC').head()

,datatype,station,attributes,value
date,,,,
2018-10-01 05:00:00+00:00,TAVG,GHCND:USW00014732,"H,,S,",21.2
2018-10-01 05:00:00+00:00,TMAX,GHCND:USW00014732,",,W,2400",25.6
2018-10-01 05:00:00+00:00,TMIN,GHCND:USW00014732,",,W,2400",18.3
2018-10-02 05:00:00+00:00,TAVG,GHCND:USW00014732,"H,,S,",22.7
2018-10-02 05:00:00+00:00,TMAX,GHCND:USW00014732,",,W,2400",26.1


We can also truncate datetimes with the `to_period()` method

In [14]:
eastern.tz_localize(None).to_period('M').index

PeriodIndex(['2018-10', '2018-10', '2018-10', '2018-10', '2018-10', '2018-10',
             '2018-10', '2018-10', '2018-10', '2018-10', '2018-10', '2018-10',
             '2018-10', '2018-10', '2018-10', '2018-10', '2018-10', '2018-10',
             '2018-10', '2018-10', '2018-10', '2018-10', '2018-10', '2018-10',
             '2018-10', '2018-10', '2018-10', '2018-10', '2018-10', '2018-10',
             '2018-10', '2018-10', '2018-10', '2018-10', '2018-10', '2018-10',
             '2018-10', '2018-10', '2018-10', '2018-10', '2018-10', '2018-10',
             '2018-10', '2018-10', '2018-10', '2018-10', '2018-10', '2018-10',
             '2018-10', '2018-10', '2018-10', '2018-10', '2018-10', '2018-10',
             '2018-10', '2018-10', '2018-10', '2018-10', '2018-10', '2018-10',
             '2018-10', '2018-10', '2018-10', '2018-10', '2018-10', '2018-10',
             '2018-10', '2018-10', '2018-10', '2018-10', '2018-10', '2018-10',
             '2018-10', '2018-10', '2018-10', '2018-

We can use the `to_timestamp()` method to convert our PeriodIndex object into a DatetimeIndex object; however, the datetimes all start at the first of the month now

In [15]:
eastern.tz_localize(None) \
    .to_period('M').to_timestamp().index

DatetimeIndex(['2018-10-01', '2018-10-01', '2018-10-01', '2018-10-01',
               '2018-10-01', '2018-10-01', '2018-10-01', '2018-10-01',
               '2018-10-01', '2018-10-01', '2018-10-01', '2018-10-01',
               '2018-10-01', '2018-10-01', '2018-10-01', '2018-10-01',
               '2018-10-01', '2018-10-01', '2018-10-01', '2018-10-01',
               '2018-10-01', '2018-10-01', '2018-10-01', '2018-10-01',
               '2018-10-01', '2018-10-01', '2018-10-01', '2018-10-01',
               '2018-10-01', '2018-10-01', '2018-10-01', '2018-10-01',
               '2018-10-01', '2018-10-01', '2018-10-01', '2018-10-01',
               '2018-10-01', '2018-10-01', '2018-10-01', '2018-10-01',
               '2018-10-01', '2018-10-01', '2018-10-01', '2018-10-01',
               '2018-10-01', '2018-10-01', '2018-10-01', '2018-10-01',
               '2018-10-01', '2018-10-01', '2018-10-01', '2018-10-01',
               '2018-10-01', '2018-10-01', '2018-10-01', '2018-10-01',
      

Alternatively, we can use the `assign()` method to handle any type conversions by passing the column names as named parameters and their new values as the value for that argument to the method call

In [16]:
df = pd.read_csv('data/nyc_temperatures.csv').rename(
        columns={'value': 'temp_C', 'attributes': 'flags'}
)

new_df = df.assign(
    date = pd.to_datetime(df.date),
    temp_F = (df.temp_C * 9/5) + 32
)

In [17]:
new_df.dtypes

date        datetime64[ns]
datatype            object
station             object
flags               object
temp_C             float64
temp_F             float64
dtype: object

In [18]:
new_df.head()

,date,datatype,station,flags,temp_C,temp_F
0,2018-10-01,TAVG,GHCND:USW00014732,"H,,S,",21.2,70.16
1,2018-10-01,TMAX,GHCND:USW00014732,",,W,2400",25.6,78.08
2,2018-10-01,TMIN,GHCND:USW00014732,",,W,2400",18.3,64.94
3,2018-10-02,TAVG,GHCND:USW00014732,"H,,S,",22.7,72.86
4,2018-10-02,TMAX,GHCND:USW00014732,",,W,2400",26.1,78.98


Additionally, we can use the `astype()` method to convert one column at a time.
 It is very common (and useful) to use `lambda` functions with `assign()`

In [19]:
df = df.assign(
    date = lambda x: pd.to_datetime(x.date),
    temp_c_whole = lambda x: x.temp_C.astype('int'),
    temp_F = lambda x: (x.temp_C * 9/5) + 32,
    temp_F_whole = lambda x: x.temp_F.astype('int')
)
df.head()

,date,datatype,station,flags,temp_C,temp_c_whole,temp_F,temp_F_whole
0,2018-10-01,TAVG,GHCND:USW00014732,"H,,S,",21.2,21,70.16,70
1,2018-10-01,TMAX,GHCND:USW00014732,",,W,2400",25.6,25,78.08,78
2,2018-10-01,TMIN,GHCND:USW00014732,",,W,2400",18.3,18,64.94,64
3,2018-10-02,TAVG,GHCND:USW00014732,"H,,S,",22.7,22,72.86,72
4,2018-10-02,TMAX,GHCND:USW00014732,",,W,2400",26.1,26,78.98,78


we don't have to know whether to convert the column into a float or an integer: we can use `pd.to_numeric()`, which will convert the data into
floats if it sees decimals.

> 
    Pandas has the ability to define columns as
    categorical; certain statistical operations both within pandas and other packages will
    be able to handle this data, provide meaningful statistics on them, and use them properly.

In [20]:
df.dtypes

date            datetime64[ns]
datatype                object
station                 object
flags                   object
temp_C                 float64
temp_c_whole             int64
temp_F                 float64
temp_F_whole             int64
dtype: object

In [21]:
df_with_categories = df.assign(
    station = df.station.astype('category'),
    datatype = df.datatype.astype('category')
)

df_with_categories.dtypes

date            datetime64[ns]
datatype              category
station               category
flags                   object
temp_C                 float64
temp_c_whole             int64
temp_F                 float64
temp_F_whole             int64
dtype: object

In [22]:
df_with_categories.describe(include='all').T

,count,unique,top,freq,mean,min,25%,50%,75%,max,std
date,93,NaN,NaN,NaN,2018-10-16 00:00:00,2018-10-01 00:00:00,2018-10-08 00:00:00,2018-10-16 00:00:00,2018-10-24 00:00:00,2018-10-31 00:00:00,NaN
datatype,93,3,TAVG,31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
station,93,1,GHCND:USW00014732,93,NaN,NaN,NaN,NaN,NaN,NaN,NaN
flags,93,2,",,W,2400",62,NaN,NaN,NaN,NaN,NaN,NaN,NaN
temp_C,93.0,NaN,NaN,NaN,15.408602,5.6,10.2,14.4,21.1,27.8,6.133703
temp_c_whole,93.0,NaN,NaN,NaN,14.956989,5.0,10.0,14.0,21.0,27.0,6.084396
temp_F,93.0,NaN,NaN,NaN,59.735484,42.08,50.36,57.92,69.98,82.04,11.040666
temp_F_whole,93.0,NaN,NaN,NaN,59.215054,42.0,50.0,57.0,69.0,82.0,11.128547


The categories we just made don't have any order to them, but pandas does support this

In [23]:
pd.Categorical(
    ['med', 'med', 'low', 'high'],
    categories = ['low', 'med', 'high'],
    ordered=True
)

['med', 'med', 'low', 'high']
Categories (3, object): ['low' < 'med' < 'high']

### Reordering, reindexing, and sorting data

Say we wanted to find the days that reached the highest temperatures in New York City during October 2018; we could sort our values by the temp_C (or temp_F) column in descending order and use head() to select the number of days we wanted to see. To accomplish this, we can use the `sort_values()` method.

In [27]:
df[df.datatype == 'TMAX']\
    .sort_values(by='temp_C', ascending=False)\
    .head(10)

,date,datatype,station,flags,temp_C,temp_c_whole,temp_F,temp_F_whole
19,2018-10-07,TMAX,GHCND:USW00014732,",,W,2400",27.8,27,82.04,82
28,2018-10-10,TMAX,GHCND:USW00014732,",,W,2400",27.8,27,82.04,82
31,2018-10-11,TMAX,GHCND:USW00014732,",,W,2400",26.7,26,80.06,80
10,2018-10-04,TMAX,GHCND:USW00014732,",,W,2400",26.1,26,78.98,78
4,2018-10-02,TMAX,GHCND:USW00014732,",,W,2400",26.1,26,78.98,78
1,2018-10-01,TMAX,GHCND:USW00014732,",,W,2400",25.6,25,78.08,78
25,2018-10-09,TMAX,GHCND:USW00014732,",,W,2400",25.6,25,78.08,78
7,2018-10-03,TMAX,GHCND:USW00014732,",,W,2400",25.0,25,77.00,77
13,2018-10-05,TMAX,GHCND:USW00014732,",,W,2400",22.8,22,73.04,73
22,2018-10-08,TMAX,GHCND:USW00014732,",,W,2400",22.8,22,73.04,73


The order in which the columns are provided will determine the sort order, with each subsequent column being used to break ties.

In [28]:
df[df.datatype == 'TMAX']\
    .sort_values(by=['temp_C', 'date'], ascending=False)\
    .head(10)

,date,datatype,station,flags,temp_C,temp_c_whole,temp_F,temp_F_whole
28,2018-10-10,TMAX,GHCND:USW00014732,",,W,2400",27.8,27,82.04,82
19,2018-10-07,TMAX,GHCND:USW00014732,",,W,2400",27.8,27,82.04,82
31,2018-10-11,TMAX,GHCND:USW00014732,",,W,2400",26.7,26,80.06,80
10,2018-10-04,TMAX,GHCND:USW00014732,",,W,2400",26.1,26,78.98,78
4,2018-10-02,TMAX,GHCND:USW00014732,",,W,2400",26.1,26,78.98,78
25,2018-10-09,TMAX,GHCND:USW00014732,",,W,2400",25.6,25,78.08,78
1,2018-10-01,TMAX,GHCND:USW00014732,",,W,2400",25.6,25,78.08,78
7,2018-10-03,TMAX,GHCND:USW00014732,",,W,2400",25.0,25,77.00,77
22,2018-10-08,TMAX,GHCND:USW00014732,",,W,2400",22.8,22,73.04,73
13,2018-10-05,TMAX,GHCND:USW00014732,",,W,2400",22.8,22,73.04,73


Since we are sorting in ascending order, in the case of a tie, the date that comes earlier in
the year will be above the later one.

we can pass `ignore_index=True` to `sort_values()` and create a fresh new index

In [29]:
df[df.datatype == 'TMAX']\
    .sort_values(by=['temp_C', 'date'], ascending=False, ignore_index=True)\
    .head(10)

,date,datatype,station,flags,temp_C,temp_c_whole,temp_F,temp_F_whole
0,2018-10-10,TMAX,GHCND:USW00014732,",,W,2400",27.8,27,82.04,82
1,2018-10-07,TMAX,GHCND:USW00014732,",,W,2400",27.8,27,82.04,82
2,2018-10-11,TMAX,GHCND:USW00014732,",,W,2400",26.7,26,80.06,80
3,2018-10-04,TMAX,GHCND:USW00014732,",,W,2400",26.1,26,78.98,78
4,2018-10-02,TMAX,GHCND:USW00014732,",,W,2400",26.1,26,78.98,78
5,2018-10-09,TMAX,GHCND:USW00014732,",,W,2400",25.6,25,78.08,78
6,2018-10-01,TMAX,GHCND:USW00014732,",,W,2400",25.6,25,78.08,78
7,2018-10-03,TMAX,GHCND:USW00014732,",,W,2400",25.0,25,77.00,77
8,2018-10-08,TMAX,GHCND:USW00014732,",,W,2400",22.8,22,73.04,73
9,2018-10-05,TMAX,GHCND:USW00014732,",,W,2400",22.8,22,73.04,73


We can use `nlargest()` to grab the n rows with the largest values according to specific criteria and `nsmallest()` to grab the n smallest rows, without the need to sort the data beforehand.

In [30]:
df[df.datatype == 'TAVG'].nlargest(n=10, columns='temp_C')

,date,datatype,station,flags,temp_C,temp_c_whole,temp_F,temp_F_whole
27,2018-10-10,TAVG,GHCND:USW00014732,"H,,S,",23.8,23,74.84,74
30,2018-10-11,TAVG,GHCND:USW00014732,"H,,S,",23.4,23,74.12,74
18,2018-10-07,TAVG,GHCND:USW00014732,"H,,S,",22.8,22,73.04,73
3,2018-10-02,TAVG,GHCND:USW00014732,"H,,S,",22.7,22,72.86,72
6,2018-10-03,TAVG,GHCND:USW00014732,"H,,S,",21.8,21,71.24,71
24,2018-10-09,TAVG,GHCND:USW00014732,"H,,S,",21.8,21,71.24,71
9,2018-10-04,TAVG,GHCND:USW00014732,"H,,S,",21.3,21,70.34,70
0,2018-10-01,TAVG,GHCND:USW00014732,"H,,S,",21.2,21,70.16,70
21,2018-10-08,TAVG,GHCND:USW00014732,"H,,S,",20.9,20,69.62,69
12,2018-10-05,TAVG,GHCND:USW00014732,"H,,S,",20.3,20,68.54,68


In [31]:
df[df.datatype == 'TAVG'].nsmallest(n=10, columns='temp_C')

,date,datatype,station,flags,temp_C,temp_c_whole,temp_F,temp_F_whole
75,2018-10-26,TAVG,GHCND:USW00014732,"H,,S,",7.3,7,45.14,45
63,2018-10-22,TAVG,GHCND:USW00014732,"H,,S,",8.3,8,46.94,46
72,2018-10-25,TAVG,GHCND:USW00014732,"H,,S,",8.8,8,47.84,47
78,2018-10-27,TAVG,GHCND:USW00014732,"H,,S,",9.4,9,48.92,48
51,2018-10-18,TAVG,GHCND:USW00014732,"H,,S,",9.6,9,49.28,49
81,2018-10-28,TAVG,GHCND:USW00014732,"H,,S,",10.2,10,50.36,50
87,2018-10-30,TAVG,GHCND:USW00014732,"H,,S,",10.2,10,50.36,50
60,2018-10-21,TAVG,GHCND:USW00014732,"H,,S,",10.7,10,51.26,51
69,2018-10-24,TAVG,GHCND:USW00014732,"H,,S,",11.0,11,51.80,51
54,2018-10-19,TAVG,GHCND:USW00014732,"H,,S,",11.3,11,52.34,52


We aren't limited to sorting values; if we wish, we can even order the columns alphabetically and sort the rows by their index values. For these tasks, we can use the
`sort_index()` method.

In [32]:
df.sample(5, random_state=0).index

Index([2, 30, 55, 16, 13], dtype='int64')

In [35]:
df.sample(5, random_state=0).sort_index().index

Index([2, 13, 16, 30, 55], dtype='int64')

When we want to target columns, we must pass in axis=1; rows will be the default (axis=0)

In [36]:
df.sort_index(axis=1).head()

,datatype,date,flags,station,temp_C,temp_F,temp_F_whole,temp_c_whole
0,TAVG,2018-10-01,"H,,S,",GHCND:USW00014732,21.2,70.16,70,21
1,TMAX,2018-10-01,",,W,2400",GHCND:USW00014732,25.6,78.08,78,25
2,TMIN,2018-10-01,",,W,2400",GHCND:USW00014732,18.3,64.94,64,18
3,TAVG,2018-10-02,"H,,S,",GHCND:USW00014732,22.7,72.86,72,22
4,TMAX,2018-10-02,",,W,2400",GHCND:USW00014732,26.1,78.98,78,26


Using `loc[]` we can specify a range of columns with similar names

In [37]:
df.loc[:, 'station':'temp_F_whole']

,station,flags,temp_C,temp_c_whole,temp_F,temp_F_whole
0,GHCND:USW00014732,"H,,S,",21.2,21,70.16,70
1,GHCND:USW00014732,",,W,2400",25.6,25,78.08,78
2,GHCND:USW00014732,",,W,2400",18.3,18,64.94,64
3,GHCND:USW00014732,"H,,S,",22.7,22,72.86,72
4,GHCND:USW00014732,",,W,2400",26.1,26,78.98,78
...,...,...,...,...,...,...
88,GHCND:USW00014732,",,W,2400",13.9,13,57.02,57
89,GHCND:USW00014732,",,W,2400",7.2,7,44.96,44
90,GHCND:USW00014732,"H,,S,",12.6,12,54.68,54
91,GHCND:USW00014732,",,W,2400",17.8,17,64.04,64


>
    Both `sort_index()` and `sort_values()` return new DataFrame objects. We must pass in inplace=True to update the dataframe we are working with.

The sort_index() method can also help us get an accurate answer when we're testing two dataframes for equality

In [38]:
df.equals(df.sort_values(by='temp_C'))

False

In [40]:
df.equals(df.sort_values(by='temp_C').sort_index())

True

If we want to use one (or more) columns as the index, we can use the `set_index()` method

In [41]:
df.set_index('date', inplace=True)
df.head()

,datatype,station,flags,temp_C,temp_c_whole,temp_F,temp_F_whole
date,,,,,,,
2018-10-01,TAVG,GHCND:USW00014732,"H,,S,",21.2,21,70.16,70
2018-10-01,TMAX,GHCND:USW00014732,",,W,2400",25.6,25,78.08,78
2018-10-01,TMIN,GHCND:USW00014732,",,W,2400",18.3,18,64.94,64
2018-10-02,TAVG,GHCND:USW00014732,"H,,S,",22.7,22,72.86,72
2018-10-02,TMAX,GHCND:USW00014732,",,W,2400",26.1,26,78.98,78


`loc[]` is optional when using ranges

In [42]:
df['2018-10-11':'2018-10-12']

,datatype,station,flags,temp_C,temp_c_whole,temp_F,temp_F_whole
date,,,,,,,
2018-10-11,TAVG,GHCND:USW00014732,"H,,S,",23.4,23,74.12,74
2018-10-11,TMAX,GHCND:USW00014732,",,W,2400",26.7,26,80.06,80
2018-10-11,TMIN,GHCND:USW00014732,",,W,2400",21.7,21,71.06,71
2018-10-12,TAVG,GHCND:USW00014732,"H,,S,",18.3,18,64.94,64
2018-10-12,TMAX,GHCND:USW00014732,",,W,2400",22.2,22,71.96,71
2018-10-12,TMIN,GHCND:USW00014732,",,W,2400",12.2,12,53.96,53


We can use the `reset_index()` method to restore the date column

In [43]:
df['2018-10-11':'2018-10-12'].reset_index()

,date,datatype,station,flags,temp_C,temp_c_whole,temp_F,temp_F_whole
0,2018-10-11,TAVG,GHCND:USW00014732,"H,,S,",23.4,23,74.12,74
1,2018-10-11,TMAX,GHCND:USW00014732,",,W,2400",26.7,26,80.06,80
2,2018-10-11,TMIN,GHCND:USW00014732,",,W,2400",21.7,21,71.06,71
3,2018-10-12,TAVG,GHCND:USW00014732,"H,,S,",18.3,18,64.94,64
4,2018-10-12,TMAX,GHCND:USW00014732,",,W,2400",22.2,22,71.96,71
5,2018-10-12,TMIN,GHCND:USW00014732,",,W,2400",12.2,12,53.96,53


Let's see what our data looks like and mark the day of the week for each row in order to understand what the index contains. We can easily isolate the date part from an index
of type `DatetimeIndex`. When isolating date parts, pandas will give us the numeric representation of what we are looking for.

In [44]:
sp = pd.read_csv(
    'data/sp500.csv', index_col='date', parse_dates=True
).drop(columns=['adj_close']) # not using this column

In [45]:
sp.head()

,high,low,open,close,volume
date,,,,,
2017-01-03,2263.879883,2245.129883,2251.570068,2257.830078,3770530000
2017-01-04,2272.820068,2261.600098,2261.600098,2270.750000,3764890000
2017-01-05,2271.500000,2260.449951,2268.179932,2269.000000,3761820000
2017-01-06,2282.100098,2264.060059,2271.139893,2276.979980,3339890000
2017-01-09,2275.489990,2268.899902,2273.590088,2268.899902,3217610000


In [46]:
sp.head()\
    .assign(day_of_week = lambda x: x.index.day_name())

,high,low,open,close,volume,day_of_week
date,,,,,,
2017-01-03,2263.879883,2245.129883,2251.570068,2257.830078,3770530000,Tuesday
2017-01-04,2272.820068,2261.600098,2261.600098,2270.750000,3764890000,Wednesday
2017-01-05,2271.500000,2260.449951,2268.179932,2269.000000,3761820000,Thursday
2017-01-06,2282.100098,2264.060059,2271.139893,2276.979980,3339890000,Friday
2017-01-09,2275.489990,2268.899902,2273.590088,2268.899902,3217610000,Monday
